<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/A(parsing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload datas

In [3]:
# upload Drain(log parser)
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/Drain.py'

--2023-07-23 10:29:52--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/Drain.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12456 (12K) [text/plain]
Saving to: ‘Drain.py’

Drain.py            100%[===================>]  12.16K  --.-KB/s    in 0s      

2023-07-23 10:29:52 (55.1 MB/s) - ‘Drain.py’ saved [12456/12456]



In [4]:
# upload HDFS dataset
!wget 'https://zenodo.org/record/3227177/files/HDFS_1.tar.gz'
!tar -xzvf "/content/HDFS_1.tar.gz" -C "/content/"   #unzip the file

--2023-07-23 10:29:55--  https://zenodo.org/record/3227177/files/HDFS_1.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161886385 (154M) [application/octet-stream]
Saving to: ‘HDFS_1.tar.gz’

HDFS_1.tar.gz       100%[===================>] 154.39M  6.67MB/s    in 2m 5s   

2023-07-23 10:32:02 (1.23 MB/s) - ‘HDFS_1.tar.gz’ saved [161886385/161886385]

HDFS.log
anomaly_label.csv


import libraries & check dataset

In [5]:
import sys
sys.path.append('/content/') #to be able for importing libarary(drain)from this address
import Drain
import pandas as pd
import numpy as np
import os
import re
from collections import OrderedDict

In [6]:
# count rows of the raw log

with open('/content/HDFS.log', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 11175629 lines


In [8]:
#Take a quick look at the first lines of the raw log:

with open('/content/HDFS.log', "r") as file:
  n = 0
  for line in file:
    print(line)
    n += 1
    if n == 3:
      break


081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010

081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906

081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010



In [9]:
#Take a quick look at the labels, which we see, its classified based on blockID

labels = pd.read_csv('/content/anomaly_label.csv')
labels.head(3)

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly


Parsing

In [ ]:
input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'HDFS.log'  # The input log file name
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Regular expression list for optional preprocessing (default: [])

regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

#keep_para=True >> we will also have ParameterList column contains parameters which are removed
parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex, keep_para=True)
parser.parse(log_file)

Parsing file: /content/HDFS.log
Total size after encoding is 11175629 11175629


In [3]:
# we upload in a case that we cant parse logs(needs time and ram)
!cp '/content/drive/MyDrive/HDFS/HDFS.log_structured.csv' '/content/Drain_result/'

In [6]:
# load structured logs
# engine='c' : The 'c' engine is faster and uses less memory than the Python engine ('python').
# If your CSV file is large, using the 'c' engine can lead to faster reading.

struct_log = pd.read_csv('/content/Drain_result/HDFS.log_structured.csv', engine='c')
struct_log.head(3)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,bbb51b95,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.19.102:54..."
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,['/mnt/hadoop/mapred/system/job_200811092030_0...
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,bbb51b95,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.10.6:4052..."


In [7]:
#uniques: unique templates of the 'EventId' column
#labels : the mapping dictionary of unique items to Numeric Representation
labels, uniques = pd.factorize(struct_log['EventId'])

print("Unique templates : \n", uniques)
print("Numeric Representation : \n", labels)


Unique templates : 
 Index(['bbb51b95', '3d91fa85', 'd38aa58d', '46003790', '5d5de21c', '44614d71',
       '9c784e29', '75627efd', '54e5f6b4', '4dec0816', '728076ac', '40651754',
       '32777b38', 'ace40671', 'e817fa45', 'bcc910df', 'dba996ef', '0567184d',
       '69bca6e5', 'd013b7a3', 'c294d20f', 'a5478aff', 'f7c33085', 'f266840a',
       '15b484af', '9621139e', 'd62e5638', '1061e5d9', '2ecc047e', '4610d0f1',
       '1a718242', 'd63ef163', '2e68ccc3', '8f2bc724', 'a333d363', 'e024fa48',
       '25382c88', '20317105', '06d16156', '78915d3a', '13eb7010', 'f79898ae',
       '124068c6', '625a2a34', 'b65fc512', 'fcd37a6d', '559305d8', 'fcf2c482'],
      dtype='object')
Numeric Representation : 
 [ 0  1  0 ... 12 12 12]


In [8]:
# Use pd.factorize() to encode the 'EventId' column
struct_log['EventId'] = labels
struct_log.head(3)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,0,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.19.102:54..."
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,1,BLOCK* NameSystem.allocateBlock: <*> <*>,['/mnt/hadoop/mapred/system/job_200811092030_0...
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,0,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.10.6:4052..."


In [ ]:
data_dict = OrderedDict()  # to search an iten faster
for idx, row in struct_log.iterrows():
    blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
    blkId_set = set(blkId_list)
    for blk_Id in blkId_set:
        if not blk_Id in data_dict:
            data_dict[blk_Id] = []
        data_dict[blk_Id].append(row['EventId'])
data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])
data_df.to_csv("HDFS_sequence.csv",index=None)

data_df.head(3)

In [ ]:
#Take a quick look at the labels, which we see, its classified based on blockID

labels = pd.read_csv('/content/anomaly_label.csv')
labels.head()

In [ ]:
# Merge the labels with the data_df DataFrame
data_df = data_df.merge(labels, on='BlockId', how='left')
data_df.head(3)


In [ ]:
# Splitting the dataset into train and test
normal_data_df = data_df[data_df['Label'] == 'Normal']

train_ratio = 0.7  # 70% for training
train_normal_len = int(len(normal_data_df) * train_ratio)

hdfs_train = normal_data_df[:train_normal_len]
hdfs_test_normal = pd.concat([normal_data_df[train_normal_len:], anomaly_data_df])

hdfs_test_abnormal = data_df[data_df['Label'] != 'Normal']

# Print the lengths of train and test datasets
print("Train Dataset Length:", len(hdfs_train))
print("Normal Test Dataset Length:", len(hdfs_test_normal))
print("Abnormal Test Dataset Length:", len(hdfs_test_abnormal))

In [ ]:
hdfs_train.head(3)

In [ ]:
hdfs_test_normal.head(3)

In [ ]:
hdfs_test_abnormal.head(3)

In [1]:
# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save datasets on google drive
hdfs_train.to_csv('/content/drive/MyDrive/hdfs_train.csv')
hdfs_test_normal.to_csv('/content/drive/MyDrive/hdfs_test_normal.csv')
hdfs_test_abnormal.to_csv('/content/drive/MyDrive/hdfs_test_abnormal.csv')